<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

import pandas,numpy,BeautifulSoup,requests

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

get data from website , i have used BeautifulSoup

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url).text
soup = BeautifulSoup(r,"html.parser")
table=soup.find('table',{'class':'wikitable'})

In [4]:

fl=0
list1=[]
list2=[]
list3=[]
for row in table.find_all("tr"):
    cols=row.find_all("td")
    if fl==0:
        pass
    else:
        list1.append(cols[0].getText().replace('\n',''))
        list2.append(cols[1].getText().replace('\n',''))
        list3.append(cols[2].getText().replace('\n',''))
    fl=1
df=pd.DataFrame(list(zip(list1,list2,list3)),columns=["Postal Code","Borough","Neighbourhood"])
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


find and remove borough which have "Not assigned"

In [5]:
dep=df[df['Borough']=="Not assigned"].index
dep

Int64Index([  0,   1,   7,  10,  15,  16,  19,  24,  25,  28,  29,  33,  34,
             35,  37,  38,  42,  43,  44,  51,  52,  53,  60,  61,  62,  69,
             70,  71,  78,  79,  87,  88,  96,  97, 101, 105, 106, 110, 115,
            118, 119, 123, 124, 125, 127, 128, 131, 132, 133, 134, 136, 137,
            140, 141, 145, 146, 149, 150, 154, 155, 158, 159, 161, 162, 163,
            164, 166, 167, 170, 171, 172, 173, 174, 175, 176, 177, 179],
           dtype='int64')

In [6]:
df=df.drop(dep,axis=0)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


split rows which have multiple Neighbourhood values

In [7]:
df['Neighbourhood']=df['Neighbourhood'].str.split(',')
df=df.explode("Neighbourhood").reset_index(drop=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor
...,...,...,...
212,M8Z,Etobicoke,Mimico NW
213,M8Z,Etobicoke,The Queensway West
214,M8Z,Etobicoke,South of Bloor
215,M8Z,Etobicoke,Kingsway Park South West


In [9]:
df.shape

(217, 3)

In [20]:
!pip install geocoder
import geocoder

You should consider upgrading via the 'c:\users\rushi\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


find latitude and longitude
i have used csv file which is provided

In [17]:
dflt=pd.read_csv("Geospatial_Coordinates.csv")
dflt

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [19]:
df1=pd.merge(df, dflt, on='Postal Code', how='inner')
df1

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
...,...,...,...,...,...
212,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
213,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
214,M8Z,Etobicoke,South of Bloor,43.628841,-79.520999
215,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999
